In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import time
import pandas as pd
import numpy as np
from fastcore.xtras import save_pickle
from sklearn.model_selection import train_test_split

from gptchem.baselines.photoswitch import train_test_photoswitch_classification_baseline
from gptchem.data import get_photoswitch_data
from gptchem.evaluator import evaluate_classification
from gptchem.gpt_classifier import MultiRepGPTClassifier
from gptchem.querier import Querier
from gptchem.tuner import Tuner

In [4]:
representation =  ["selfies", "deepsmiles", "canonical", "inchi", "tucan", "iupac_name"]

In [5]:
test_data = pd.read_csv("../../02_multirep/solubility_test_data.csv")
train_data = pd.read_csv("../../02_multirep/esol_data.csv")

test = test_data.dropna(subset=representation + ["measured log(solubility:mol/L)"])
train = train_data.dropna(subset=representation + ["measured log(solubility:mol/L)"])

In [7]:
train["binned"], bins = pd.qcut(
    train["measured log(solubility:mol/L)"],
    5,
    labels=np.arange(5),
    retbins=True,
)
test["binned"] = pd.cut(
    test["measured log(solubility:mol/L)"], bins=bins, labels=np.arange(5)
)

/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ipykernel_10384/3014218382.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["binned"], bins = pd.qcut(
/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ipykernel_10384/3014218382.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["binned"] = pd.cut(


In [8]:
test

,Unnamed: 0.1,Unnamed: 0,Chemical name,Crystal structure CSD refcode,LogS exp (mol/L),Reference,Test,SMILES,Smiles Source,Chemspider reference number,...,tucan_y,iupac_name_y,smiles,selfies,deepsmiles,canonical,inchi,tucan,iupac_name,binned
0,0,1,Adenosine,ADENOS10,-1.73,2,False,Nc1ncnc2c1ncn2C1OC(CO)C(O)C1O,ChemSpider,54923,...,C10N5O4/(1-4)(1-5)(1-12)(2-13)(2-14)(3-12)(3-1...,2-(6-aminopurin-9-yl)-5-(hydroxymethyl)oxolane...,Nc1ncnc2c1ncn2C1OC(CO)C(O)C1O,['[N][C][=N][C][=N][C][=C][Ring1][=Branch1][N]...,Ncncncc6ncn5COCCO))CO)C5O,Nc1ncnc2c1ncn2C1OC(CO)C(O)C1O,InChI=1S/C10H13N5O4/c11-8-5-9(13-2-12-8)15(3-1...,C10N5O4/(1-4)(1-5)(1-12)(2-13)(2-14)(3-12)(3-1...,2-(6-aminopurin-9-yl)-5-(hydroxymethyl)oxolane...,3
1,1,2,Allopurinol,ALOPUR,-2.26,8,False,Oc1ncnc2[nH]ncc12,ChemSpider,2010,...,C5N4O/(1-2)(1-4)(1-5)(2-8)(3-6)(3-7)(4-6)(4-9)...,"1,5-dihydropyrazolo[3,4-d]pyrimidin-4-one",Oc1ncnc2[nH]ncc12,['[O][C][=N][C][=N][C][NH1][N][=C][C][Ring1][=...,Ocncnc[nH]ncc95,Oc1ncnc2[nH]ncc12,InChI=1S/C5H4N4O/c10-5-3-1-8-9-4(3)6-2-7-5/h1-...,C5N4O/(1-2)(1-4)(1-5)(2-8)(3-6)(3-7)(4-6)(4-9)...,"1,5-dihydropyrazolo[3,4-d]pyrimidin-4-one",3
2,2,3,Trimethoprim,AMXBPM10,-2.95,3,False,COc1cc(Cc2cnc(N)nc2N)cc(OC)c1OC,ChemSpider,953,...,C14N4O3/(1-4)(1-5)(2-4)(2-12)(3-4)(3-13)(5-6)(...,"5-[(3,4,5-trimethoxyphenyl)methyl]pyrimidine-2...",COc1cc(Cc2cnc(N)nc2N)cc(OC)c1OC,['[C][O][C][=C][C][Branch1][=C][C][C][=C][N][=...,COcccCccncN)nc6N))))))))ccOC))c6OC,COc1cc(Cc2cnc(N)nc2N)cc(OC)c1OC,InChI=1S/C14H18N4O3/c1-19-10-5-8(6-11(20-2)12(...,C14N4O3/(1-4)(1-5)(2-4)(2-12)(3-4)(3-13)(5-6)(...,"5-[(3,4,5-trimethoxyphenyl)methyl]pyrimidine-2...",2
3,3,5,Benzoic acid,BENZAC02,-1.58,8,False,O=C(O)c1ccccc1,ChemSpider,5376,...,C7O2/(1-2)(1-3)(2-4)(3-5)(4-6)(5-6)(6-7)(7-8)(...,benzoic acid,O=C(O)c1ccccc1,['[O][=C][Branch1][C][O][C][=C][C][=C][C][=C][...,O=CO)cccccc6,O=C(O)c1ccccc1,"InChI=1S/C7H6O2/c8-7(9)6-4-2-1-3-5-6/h1-5H,(H,...",C7O2/(1-2)(1-3)(2-4)(3-5)(4-6)(5-6)(6-7)(7-8)(...,benzoic acid,3
4,4,6,Salbutamol,BHHPHE,-1.22,2,False,CC(C)(C)NCC(O)c1ccc(O)c(CO)c1,ChemSpider,2121,...,C13NO3/(1-10)(2-10)(3-10)(4-6)(4-7)(5-7)(5-8)(...,4-[2-(tert-butylamino)-1-hydroxyethyl]-2-(hydr...,CC(C)(C)NCC(O)c1ccc(O)c(CO)c1,['[C][C][Branch1][C][C][Branch1][C][C][N][C][C...,CCC)C)NCCO)ccccO)cCO))c6,CC(C)(C)NCC(O)c1ccc(O)c(CO)c1,"InChI=1S/C13H21NO3/c1-13(2,3)14-7-12(17)9-4-5-...",C13NO3/(1-10)(2-10)(3-10)(4-6)(4-7)(5-7)(5-8)(...,4-[2-(tert-butylamino)-1-hydroxyethyl]-2-(hydr...,3
5,5,7,Quinidine,BOMDUC,-2.81,2,False,C=CC1CN2CCC1CC2C(O)c1ccnc2ccc(OC)cc12,ChemSpider,1909,...,C20N2O2/(1-2)(2-10)(3-12)(3-13)(4-9)(4-14)(5-7...,(5-ethenyl-1-azabicyclo[2.2.2]octan-7-yl)-(6-m...,C=CC1CN2CCC1CC2C(O)c1ccnc2ccc(OC)cc12,['[C][=C][C][C][N][C][C][C][Ring1][=Branch1][C...,C=CCCNCCC6CC6CO)cccnccccOC))cc%106,C=CC1CN2CCC1CC2C(O)c1ccnc2ccc(OC)cc12,InChI=1S/C20H24N2O2/c1-3-13-12-22-9-7-14(13)10...,C20N2O2/(1-2)(2-10)(3-12)(3-13)(4-9)(4-14)(5-7...,(5-ethenyl-1-azabicyclo[2.2.2]octan-7-yl)-(6-m...,2
6,6,10,(RS)-Atenolol,CEZVIN,-1.21,4,False,CC(C)NCC(O)COc1ccc(CC(N)=O)cc1,ChemSpider,389880,...,C14N2O3/(1-10)(2-10)(3-5)(3-8)(4-6)(4-8)(5-13)...,2-[4-[2-hydroxy-3-(propan-2-ylamino)propoxy]ph...,CC(C)NCC(O)COc1ccc(CC(N)=O)cc1,['[C][C][Branch1][C][C][N][C][C][Branch1][C][O...,CCC)NCCO)COccccCCN)=O)))cc6,CC(C)NCC(O)COc1ccc(CC(N)=O)cc1,InChI=1S/C14H22N2O3/c1-10(2)16-8-12(17)9-19-13...,C14N2O3/(1-10)(2-10)(3-5)(3-8)(4-6)(4-8)(5-13)...,2-[4-[2-hydroxy-3-(propan-2-ylamino)propoxy]ph...,3
7,7,11,Chloral Hydrate,CHORLH01,1.70,2,False,OC(O)C(Cl)(Cl)Cl,ChemSpider,2241,...,C2Cl3O2/(1-2)(1-3)(1-4)(2-5)(2-6)(2-7),"2,2,2-Trichloroethane-1,1-diol",OC(O)C(Cl)(Cl)Cl,['[O][C][Branch1][C][O][C][Branch1][C][Cl][Bra...,OCO)CCl)Cl)Cl,OC(O)C(Cl)(Cl)Cl,"InChI=1S/C2H3Cl3O2/c3-2(4,5)1(6)7/h1,6-7H",C2Cl3O2/(1-2)(1-3)(1-4)(2-5)(2-6)(2-7),"2,2,2-Trichloroethane-1,1-diol",NaN
9,9,14,Chlorprothixene,CMAPTX,-6.75,3,False,CN(C)CCC=C1c2ccccc2Sc2ccc(Cl)cc21,ChemSpider,2606,...,C18ClNS/(1-2)(1-4)(2-6)(3-5)(3-10)(4-11)(5-15)...,"3-(2-chlorothioxanthe